# Tutorial 3: Reinforcement learning across temporal scales

**Week 1, Day 2: Comparing Tasks**

**By Neuromatch Academy**

__Content creators:__ Leila Wehbe & Swapnil Kumar

__Content reviewers:__ Samuele Bolotta, Lily Chamakura, RyeongKyung Yoon, Yizhou Chen, Ruiyi Zhang

__Production editors:__ Konstantine Tsafatinos, Ella Batty, Spiros Chavlis, Samuele Bolotta, Hlib Solodzhuk

<br>


___


# Tutorial Objectives

*Estimated timing of tutorial: 20-30min*

By the end of this tutorial, participants will be able to:


- Review how a reinforcement learning algorithm can be evaluated.
- Formulate a simple meta-reinforcement learning framework where an algorithm learns how to learn in different episodes.
- Investigate learning at different temporal scales.


In [ ]:
# @title Tutorial slides
# @markdown These are the slides for the videos in all tutorials today

from IPython.display import IFrame
from ipywidgets import widgets
out = widgets.Output()

link_id = "x4pa5"

with out:
    print(f"If you want to download the slides: https://osf.io/download/{link_id}/")
    display(IFrame(src=f"https://mfr.ca-1.osf.io/render?url=https://osf.io/{link_id}/?direct%26mode=render%26action=download%26mode=render", width=730, height=410))
display(out)

---
# Setup



##  Install and import feedback gadget


In [ ]:
# @title Install and import feedback gadget

!pip install vibecheck requests matplotlib ipython numpy --quiet

from vibecheck import DatatopsContentReviewContainer
def content_review(notebook_section: str):
    return DatatopsContentReviewContainer(
        "",  # No text prompt
        notebook_section,
        {
            "url": "https://pmyvdlilci.execute-api.us-east-1.amazonaws.com/klab",
            "name": "neuromatch_neuroai",
            "user_key": "wb2cxze8",
        },
    ).render()


feedback_prefix = "W1D2_T3"

In [ ]:
# @title Import dependencies
# @markdown

# Standard library imports
import logging
import hashlib
import os

# Third-party imports
import requests
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
from IPython.display import IFrame, display, Image
import numpy as np

# Set up logging
logging.basicConfig(level=logging.INFO)

## Data retrieval

In [ ]:
# URL of the file on OSF
url = "https://osf.io/swrmj/download"

# Send GET request
response = requests.get(url)

# Save the response content to a file
with open('trials.py', 'wb') as f:
    f.write(response.content)

# Execute the downloaded Python file
%run trials.py

##  Figure settings


In [ ]:
# @title Figure settings
# @markdown

logging.getLogger('matplotlib.font_manager').disabled = True

%matplotlib inline
%config InlineBackend.figure_format = 'retina' # perform high definition rendering for images and plots
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/main/nma.mplstyle")

# Section 1: Reinforcement learning across temporal scales

## Outline:

This tutorial:
- Reviews the basics of reinforcement learning
- Introduces meta-reinforcement learning (meta-RL)
- Studies how "learning to learn" is implemented in the context of a binary bandit task
- Studies how learning is affected by changing temporal scales

##  Video 1: meta-RL


In [ ]:
# @title Video 1: meta-RL

from ipywidgets import widgets
from IPython.display import YouTubeVideo
from IPython.display import IFrame
from IPython.display import display

class PlayVideo(IFrame):
  def __init__(self, id, source, page=1, width=400, height=300, **kwargs):
    self.id = id
    if source == 'Bilibili':
      src = f'https://player.bilibili.com/player.html?bvid={id}&page={page}'
    elif source == 'Osf':
      src = f'https://mfr.ca-1.osf.io/render?url=https://osf.io/download/{id}/?direct%26mode=render'
    super(PlayVideo, self).__init__(src, width, height, **kwargs)

def display_videos(video_ids, W=400, H=300, fs=1):
  tab_contents = []
  for i, video_id in enumerate(video_ids):
    out = widgets.Output()
    with out:
      if video_ids[i][0] == 'Youtube':
        video = YouTubeVideo(id=video_ids[i][1], width=W,
                             height=H, fs=fs, rel=0)
        print(f'Video available at https://youtube.com/watch?v={video.id}')
      else:
        video = PlayVideo(id=video_ids[i][1], source=video_ids[i][0], width=W,
                          height=H, fs=fs, autoplay=False)
        if video_ids[i][0] == 'Bilibili':
          print(f'Video available at https://www.bilibili.com/video/{video.id}')
        elif video_ids[i][0] == 'Osf':
          print(f'Video available at https://osf.io/{video.id}')
      display(video)
    tab_contents.append(out)
  return tab_contents

video_ids = [('Youtube', 'BOGX0KvpnEU')]
tab_contents = display_videos(video_ids, W=730, H=410)
tabs = widgets.Tab()
tabs.children = tab_contents
for i in range(len(tab_contents)):
  tabs.set_title(i, video_ids[i][0])
display(tabs)

## Reinforcement learning review

Reinforcement learning is a learning paradigm where an agent interacts with an environment and learns a policy to maximize its cumulative rewards. We focus here on a simple environment: a binary "two-armed bandit" (Figure A), a name for a gambling machine found in casinos (because it steals your money). In this task, at each opportunity the agent chooses arm $i$ (either Left $L$ or Right $R$) and gets a reward with probability $p_i$ that is unknown to the agent. On each trial, only one of the arms will return a reward, so $p_L + p_R = 1$ (which the agent does know).

<img src="https://github.com/neuromatch/NeuroAI_Course/blob/main/tutorials/W1D2_ComparingTasks/static/two_armed_bandit.png?raw=true" width=600 />

*Figure A. Example of binary bandit. The left arm has a probability $p_L$ of giving a reward, and the right arm a probability $p_R = 1-p_L$ of giving a reward. At a given trial, the agent pulls one of the arms and receives a reward with the corresponding probability.*

The aim of the agent is to maximize its cumulative reward over a series of trials (e.g., 100 trials). This would correspond to correctly identifying the arm that returns more reward (has a higher probability) and choosing it for subsequent trials once it's sure about its choice. Thus, while solving this problem, the agent will need to balance exploration and exploitation. At a high level, more exploration is useful while the agent is not certain of which arm has more reward, and once the agent is more certain, it is more advantageous to exploit the arm that has a higher probability of reward.

There are different methods that are usually used to solve this problem. An optimal method to solve some instances of bandit problems is [Gittin indices](https://www.statslab.cam.ac.uk/~rrw1/oc/ocgittins.pdf). This method incorporates the estimated rewards and their uncertainty, and involves solving an expensive dynamical programing problem. The simplest method is [epsilon greedy](https://medium.com/@ym1942/exploring-multi-armed-bandit-problem-epsilon-greedy-epsilon-decreasing-ucb-and-thompson-02ad0ec272ee), where exploration and exploitation are balanced by choosing a random action with probability $\epsilon$ and the best known action with probability $1-\epsilon$. In the [Upper Confidence Bound (UCB)](https://medium.com/@ym1942/exploring-multi-armed-bandit-problem-epsilon-greedy-epsilon-decreasing-ucb-and-thompson-02ad0ec272ee), the choice of action also incorporates both the estimated rewards and the estimated uncertainty of the rewards. In [Thompson sampling](https://medium.com/@ym1942/exploring-multi-armed-bandit-problem-epsilon-greedy-epsilon-decreasing-ucb-and-thompson-02ad0ec272ee), the choice is made according to a Bayesian model that estimates the posterior probabilities of the reward.

### Discussion point 1
Imagine training an RL algorithm on the binary bandit problem above, with a method that depends on uncertainty about the reward probabilities (like UCB). Assume that $p_L=0.05$ and $p_R=0.95$. How rapidly will the agent become certain of its estimates of the expected reward of each arm? Do you expect the exploration phase to take a long time or a short time?

What about the case when $p_L=0.51$ and $p_R=0.49$? How rapidly will the agent become certain of its estimates, and how will that affect the exploration phase?

#### Answer
When one of the arms has a high reward, it is easier to identify it. The uncertainty of the agent thus reduces quickly, and they spend less time in exploration. When the probabilities of rewards are more equal (close to 0.5), a lot of trials are needed to reduce the uncertainty, and the exploration phase lasts a long time.

### Evaluating decision-making strategies

One key metric for evaluating an agent's strategies is to compute the cumulative regret metric. Regret at a specific time step refers to the difference in reward between the chosen strategy and the reward that could have been obtained by selecting the best possible action. The cumulative regret refers to the sum of the regret from the start of the episode until the current time step $T$. For our example of binary bandit, it corresponds to:

$$R(T) = \sum_{t = 1}^T (p^* - \mathbb{E}(p_{a_t})),$$

where $p^*$ is the probability of the reward for the best arm, i.e., max($p_L$, $p_r$). $\mathbb{E}(p_{a_t})$ corresponds to the expected probability of reward for the action that was chosen at previous time $t$.

### Discussion point 2

Consider the setting where $p_L=0.25$ and $p_R=0.75$. Let's derive a lower bound and an upper bound for cumulative regret over 100 trials of one episode.

- Plot the lower bound of the cumulative regret, i.e. the cumulative regret in the best case scenario where the agent choses the optimal arm at every trial, plotted against the number of trials.
- In the same figure, plot the upper bound of the cummulative regret, i.e. the cumulative regret in the worst case scenario where the agent choses the non-optimal arm at every trial, plotted against the number of trials.

#### Answer

In [ ]:
t = np.arange(1,101) # Array representing trials from 1 to 100
### In the best case scenario, regret is 0 at every trial, resulting in cumulative regret of zero
cr_lb = np.zeros(100)
### In the worst case scenario, regret is 0.5 at every trial
regret_up = np.zeros(100) + 0.5
cr_up = np.cumsum(regret_up)

plt.plot(t,cr_lb, label = 'lower bound')
plt.plot(t,cr_up, label = 'upper bound')

plt.xlabel('trial')
plt.ylabel('cumulative regret')
plt.legend()

# Section 2: Meta-learning and meta-reinforcement learning

Meta-learning is a sub-field of machine learning that corresponds to a process in which an algorithm "learns to learn". The idea is that the algorithm will learn during training how to adapt itself to different tasks. When faced with a new task, the trained algorithm will quickly adapt to this new task without requiring additional training (the weights of the algorithm are not changed).

In meta-reinforcement learning (meta-RL), this concept is applied to a reinforcement learning agent. During training, the agent is exposed to different episodes with different environment, and gains meta-learning strategies. At deployment, the agent is faced with a new environment and uses the trials from that episode (the combination of actions, states and rewards) to learn a policy on the fly. Meta-RL algorithms are typically implemented using a deep learning architecture such as an LSTM. Once they are trained, the weights of the LSTM are fixed. When faced with a new episode, the activations in the LSTM implement the nested learning in which the algorithm identifies the best arm for this specific episode.

## Prefrontal cortex as a meta-reinforcement learning system

In recent years, the field of neuroAI has opened new avenues for understanding and developing AI systems that emulate the complexity and efficiency of the human brain to better model the human brain or to gain new functionality in AI systems. An important work in this domain is the following paper:

[1] Wang, Jane X., Zeb Kurth-Nelson, Dharshan Kumaran, Dhruva Tirumala, Hubert Soyer, Joel Z. Leibo, Demis Hassabis, and Matthew Botvinick. **"Prefrontal cortex as a meta-reinforcement learning system."** Nature neuroscience 21, no. 6 (2018): 860-868.

Available at: https://www.nature.com/articles/s41593-018-0147-8

This paper aims to reconciliate two seemingly disparate ideas about learning in the prefrontal cortex (PFC). The activations in the PFC have been shown to be correlated to actions, values and rewards, indicating that the PFC is engaged in RL. Another set of findings, including the fact that PFC activity is correlated with the recent history of rewards and actions, also suggests that the PFC is acting as a meta-RL agent. The paper proposes a model of the PFC, the weights of which are optimized during training to allow it to learn a second RL algorithm dynamically, as the properties of the environment changes. The second RL algorithm is implemented by the activations of the PFC, and is independent of the original training task. This allows the PFC to adapt to new situations using prior learning experiences, essentially learning to learn.

We will explore here the demo in Figure 1 of the paper. Figure C belows shows the architecture of the meta-RL model, which is implemented as an LSTM. We will explore how, after training, the weights of the LSTM implement an RL procedure (e.g., we will see if this procedure deals with the exploration-exploitation tradeoff).

We will also explore how learning can occur at different time scales (see figure B below):

**Latent world state (fast)**: In this demo, at the start of each episode, a new set of probabilities $p_L$ and $p_R = 1-p_L$ are sampled. This corresponds to latent world state parameters that change on a slower time scale than the observations level (i.e., trials), but still at a relatively fast scale (each episode here has 100 trials).

**Context (slow)**: We think of the context level as a slower time scale at which the world is changing and affecting how the parameters of the latent world state are sampled. In this paper, the authors train their meta-RL model in two contexts. In the first (independent bandit context), the parameters of the successive latent world state (i.e., $p_L$ and $p_R$ from two successive episodes) are not correlated. There is no specific knowledge about the parameters that can be generalized from one episode to the next. In the second (correlated bandit context), the parameters are anti-correlated. Thus, there is information in the parameters of the current episode that the model can use for faster learning in the next episode. We will explore how training in these two different contexts affects performance.

<img src="https://github.com/neuromatch/NeuroAI_Course/blob/main/tutorials/W1D2_ComparingTasks/static/learning_temporal_scales.png?raw=true" width=600 />

*Figure B - Learning across temporal scales*

### Architecture

<img src="https://github.com/neuromatch/NeuroAI_Course/blob/main/tutorials/W1D2_ComparingTasks/static/model_architecture.png?raw=true" width=600 />

*Figure C - Model architecture. Adapted from Figure 1 of Wang et al. The PFC, along with the basal ganglia and directly connected thalamus, are modeled as a recurrent network. The synaptic weights are adjusted during training. a = action, r = reward, v = state value. Specifically, the algorithm is implemented as an LSTM (part b above shows the model in more detail, along with an inset of one LSTM model unit illustrating the maintenance mechanism).*


### Training an agent in an uncorrelated setting - exploration/exploitation tradeoff

We first focus on showing how the meta-RL algorithm implements an RL procedure. To do so, we look at how this procedure balances exploration and exploitation under different episode parameters.

In figure D, you can see the chosen actions of the LSTM agent across 101 different episodes. The episodes span the range of probabilities $p_L \in \{0,0.01,0.02,...,0.99,1\}$. Each row corresponds to an episode, and the episodes are ordered in terms of the reward probability of the left arm. For each episode, the action taken by the agent in the 100 trials are shown. In the top half of the matrix, the optimal choice is the left arm (Arm 1), and in the bottom half, the optimal choice is the right arm (Arm 2). Here, the agent is trained in the independent bandit context.

<img src="https://github.com/neuromatch/NeuroAI_Course/blob/main/tutorials/W1D2_ComparingTasks/static/exploration_exploitation_tradeoff.png?raw=true" width=600 />

*Figure D - Exploration exploitation tradeoff. From the https://github.com/nathanwispinski/meta-rl repo which adapts the Wang et al. model. The episodes are ranked on the y-axis according to the probability of the left arm. The x-axis corresponds to the 100 successive trials in one episode. The color indicates the choice of the agent. In the lower half of the square, the right arm (Arm 2) is optimal, and in the upper half, the left arm (Arm 1) is optimal.*



### Discussion point 3

Observe the actions of the model for different episodes. How do they vary as the probabilities vary? What sets of probabilities are typically easy to learn? And which are harder to learn?

In the cells below, you can find an array in which each row corresponds to a row of Figure D. Group the rows into groups of 25 rows and average them, so you get the average arm that is selected at each time point. How do these curves showcase a longer vs shorter exploration phase?

#### Answer
The transition from exploration to exploitation occurs more slowly in more difficult problems, as indicated in the provided figure D. In episodes with reward parameters such as 0.6 and 0.4, where the difference in reward probability between actions is smaller, it's tougher for the agent to ascertain which action is better. Hence, more exploration is needed before a confident shift to exploitation can occur.

In contrast, with a larger discrepancy in rewards (e.g., 0.75 vs. 0.25), the agent can more quickly determine the more rewarding action and thus switch to exploitation sooner.

Interestingly, in these examples, there are still instances in which 100 trials are not enough for the algorithm to converge on the correct arm even when $p_L$ is relatively small, and some in which the algorithm converges quickly to the right arm even though $p_L$ is close to 0.5. For a more robust estimation of the average duration of exploration, it would be useful to look at the behavior of the network for many episodes while using the same parameters.

### Loading image info

In [ ]:
normalize = mcolors.Normalize(vmin=0, vmax=11)
colormap = cm.jet

print(Trials.shape)

for i in np.arange(0,100,25):
    plt.plot(Trials[i:i+25].mean(0), color = colormap(normalize(i/10)),
             label = 'p_L = {:.2f} - {:.2f}'.format(i*0.01,(i+25)*0.01))
plt.legend()

### Uncorrelated vs. correlated settings, seen through cumulative regret
In the paper, the average cumulative regret (defined above) of the meta-RL agent under the independent bandit and the correlated bandits contexts is computed over 300 episodes for the setting $p_L =0.25$ and $p_R = 0.75$ (see figure E). It is compared to the cumulative regret for three common algorithms (Gittin indices, Thompson sampling and UCB, also discussed above).


<img src="https://github.com/neuromatch/NeuroAI_Course/blob/main/tutorials/W1D2_ComparingTasks/static/cummulative_regret.png?raw=true" width=600 />

*Figure E - Cumulative Regret tested on a probability setting of 0.25, 0.75. Adapted from Figure 1 of Wang et al.*

Notice that the meta-RL agent trained in the correlated bandit context is able to capitalize on the information present in successive episodes and obtain a better performance than the one trained in the independent bandit context, matching the optimal performance of the Gittins indices algorithm. Both meta-RL algorithms perform better than the Thompson sampling and UCB algorithms.

### The learned model dynamics implement an RL algorithm

We explore here in more depth how the activations in the LSTM implement the RL algorithm, by visualizing the top principal components (PCs) projections of the internal states of the LSTM during the different trials of an episode. Specifically, we look at the meta-RL agent trained in the correlated bandit setting (Figure F).

<img src="https://github.com/neuromatch/NeuroAI_Course/blob/main/tutorials/W1D2_ComparingTasks/static/evolution.png?raw=true" width=600 />

*Figure F - Evolution of model activations during different episodes. Adapted from Figure 1 of Wang et al. Each subfigure is a different episode, with the arm probabilities shown at the top. The activations at the 100 trials of an episode are projected to their top two PCs and scatterplotted using an asterix when the action decided by the model is left and a triangle when the action is right. The color indicates the number of a trial.  Adapted from Figure 1 of Wang et al.*

### Discussion point 4
In Figure F, the PCs of the model activations are obtained and the 100 trials in four episodes with specific probabilities are shown, projected onto PC1 and PC2. Observe the different patterns:
- Which two episodes show a fast convergence to an optimal action? How does this manifest?
- In the other two episodes, describe the trajectory of the activations, does the model change its estimation of the optimal arm? What leads to this change?
- What does the PC1 dimension appear to represent?


### A hint

The adaptation to fast-changing latent variables, as described in the changes in activation patterns over trials, shows the algorithm's capability to adjust its internal representations based on the feedback from the environment.

- The first and last episode show a very fast convergence towards the optimal arm. The trajectory is very fast towards one or the other side of the PC space.

- In the middle two episodes, it takes longer to converge (they also correspond to more difficult settings). The model actually first decide on left, but then after sampling a right action, end up converging on right.

- The first PC appears to correspond to the certainty of the algorithm about the optimal arm (low PC value = left arm, high PC value = right arm).
